In [6]:
import requests
from datetime import datetime
import os
import pandas as pd
import numpy as np


<font color=#1DA1F2>

# Individual Provider Attributes

In [7]:
df=pd.read_csv('data/npidata_pfile_20240805-20240811.csv',low_memory=False)
# let's only identify attributes relevant for individual providers :
df=df[['NPI','Entity Type Code','Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name',  'Provider Credential Text', 
 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address', 'Provider Business Practice Location Address City Name',
 'Provider Business Practice Location Address State Name', 'Provider Business Practice Location Address Postal Code', 'Provider Business Practice Location Address Telephone Number',
 'Provider Enumeration Date', 'Last Update Date', 'NPI Deactivation Reason Code', 'NPI Deactivation Date', 'NPI Reactivation Date', 'Provider Gender Code', 'Healthcare Provider Taxonomy Code_1',
 'Provider License Number_1', 'Provider License Number State Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Taxonomy Code_2', 'Provider License Number_2',
 'Provider License Number State Code_2', 'Healthcare Provider Primary Taxonomy Switch_2',  'Certification Date'
 ]]
df.sample(3)

# df = spark.read.format("csv").option("header","true").load("Files/Provider/npidata_pfile_20240805-20240811.csv")
# # df now is a Spark DataFrame containing CSV data from "Files/Provider/npidata_pfile_20240805-20240811.csv".
# # display(df)

,NPI,Entity Type Code,Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,...,Provider Gender Code,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Certification Date
12686,1972034387,1.0,STEPHENSON,KRISTA,NaN,NaN,2401 S 31ST ST,NaN,TEMPLE,TX,...,F,208600000X,V1273,TX,N,2086X0206X,V1273,TX,Y,08/07/2024
6610,1063599009,1.0,SULLIVAN,THERESA,M.,APRN,1515 HOLCOMBE BLVD,NaN,HOUSTON,TX,...,F,364S00000X,AP103697,TX,Y,NaN,NaN,NaN,NaN,08/06/2024
12482,1396340717,1.0,GIANG,TOMMY,NaN,NaN,7000 W BUTLER PIKE,NaN,AMBLER,PA,...,M,183500000X,RP454068,PA,Y,NaN,NaN,NaN,NaN,08/07/2024


In [9]:
# Geocode the addresses using Azure Maps API
def get_geocode_and_clean_address(street, city, state, zip_code, azure_maps_api_key):
    # Construct the full address
    address = f"{street}, {city}, {state} {zip_code}"
    
    # print(f"Original address: {address}") # Print the original address

    # The base URL for the Azure Maps Search Address API
    url = 'https://atlas.microsoft.com/search/address/json'

    # Parameters for the request
    params = {
        'api-version': '1.0',
        'subscription-key': azure_maps_api_key,
        'query': address
    }

    # Make the request to the Azure Maps Search Address API
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract the first result from the response
        if 'results' in data and len(data['results']) > 0:
            result = data['results'][0]
            address = result['address']
            position = result['position']
            return address['freeformAddress'], position['lat'], position['lon']
        else:
            return None, None, None
    else:
        return None, None, None


In [11]:
# for each row in the dataframe, get the geocode and clean address and add the new columns to the dataframe, clean_address, latitude, longitude
df['clean_address'] = np.nan
df['latitude'] = np.nan
df['longitude'] = np.nan
bing_maps_api_key =os.environ["bing_maps_api_key"]

for index, row in df.iterrows():
    clean_address, latitude, longitude = get_geocode_and_clean_address(row['Provider First Line Business Practice Location Address'], row['Provider Business Practice Location Address City Name'], row['Provider Business Practice Location Address State Name'], row['Provider Business Practice Location Address Postal Code'], bing_maps_api_key)
    df.at[index, 'clean_address'] = clean_address
    df.at[index, 'latitude'] = latitude
    df.at[index, 'longitude'] = longitude

df.sample(3)

Original address: 414 PLAZA DR STE 301, WESTMONT, IL 605595508


C:\Users\BinyamWoemago\AppData\Local\Temp\ipykernel_1976\412323831.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '414 Plaza Drive, Westmont, IL 60559' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'clean_address'] = clean_address


Original address: 1441 GUTHRIE DR NW STE 106, CLEVELAND, TN 373113642
Original address: 4550 HARRISON BLVD, OGDEN, UT 844033113
Original address: 11027 BURBANK BLVD, NORTH HOLLYWOOD, CA 916012431
Original address: 784 S CLEARWATER LOOP STE R, POST FALLS, ID 838549599
Original address: 220 E HACIENDA AVE, CAMPBELL, CA 950086617
Original address: 620 MICHIGAN AVE NE, WASHINGTON, DC 200643839
Original address: 100 N HOWARD ST STE R, SPOKANE, WA 992010508
Original address: 4430 MISSOURI AVE # 1267, FORT LEONARD WOOD, MO 654739098
Original address: 2143 F ST, EUREKA, CA 955013738
Original address: 401 N CENTRAL AVE, INVERNESS, FL 344533838
Original address: 12246 QUEENSTON BLVD STE G, HOUSTON, TX 770955355
Original address: 3000 UNITED FOUNDERS BLVD STE 103, OKLAHOMA CITY, OK 731124294
Original address: 1215 21ST AVE S, NASHVILLE, TN 372322020
Original address: 2557 CHINO HILLS PKWY STE C, CHINO HILLS, CA 917095603
Original address: 2557 CHINO HILLS PKWY STE C, CHINO HILLS, CA 917095603
Ori

,NPI,Entity Type Code,Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,...,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Certification Date,clean_address,latitude,longitude
16805,1114758653,1.0,TOUART,KELLY,LAFLEUR,FNP-C,1515 RIVER RD,NaN,JEFFERSON,LA,...,LA,Y,NaN,NaN,NaN,NaN,08/07/2024,"1515 River Road, New Orleans, LA 70121",29.95976,-90.14666
25544,1790172104,2.0,NaN,NaN,NaN,NaN,8327 BRIMHALL RD STE 703,NaN,BAKERSFIELD,CA,...,NaN,Y,NaN,NaN,NaN,NaN,08/09/2024,"8327 Brimhall Road, Bakersfield, CA 93312",35.36733,-119.09595
13325,1992751150,1.0,DAVIS,JERRY,B,MD,320 WHITTINGTON PKWY,SUITE 301,LOUISVILLE,KY,...,IN,N,207L00000X,26729,KY,Y,08/07/2024,"320 Whittington Parkway, Louisville, KY 40222",38.24176,-85.57873


In [12]:
df.to_csv('data/npidata_pfile_20240805-20240811_geocoded.csv', index=False)

In [15]:
df[['NPI','Provider Last Name (Legal Name)','Provider First Name','Provider First Line Business Practice Location Address', 'Provider Business Practice Location Address City Name','clean_address',  'latitude','longitude']].sample(20)

,NPI,Provider Last Name (Legal Name),Provider First Name,Provider First Line Business Practice Location Address,Provider Business Practice Location Address City Name,clean_address,latitude,longitude
25403,1770329161,GREENFIELD,JENNIFER,9015 MURRAY AVE STE 100,GILROY,"9015 Murray Avenue, Gilroy, CA 95020",37.03091,-121.57471
18101,1942827878,FLORES,LEANDRA,1500 ROYAL PALM SQUARE BLVD STE 105,FORT MYERS,"1500 Royal Palm Square Boulevard, Fort Myers, ...",26.59381,-81.88600
21876,1265263578,ARMSTRONG,ELIZABETH,2521 N ELMS RD,FLUSHING,"2521 North Elms Road, Flushing, MI 48433",43.04524,-83.81419
7817,1598971871,RENDON,ABEL,11306 MOUNTAIN VIEW AVE STE A-100,LOMA LINDA,"11306 Mountain View Avenue, Loma Linda, CA 92354",34.04751,-117.24480
3746,1619710647,GUNDUZ,MEHMET,736 CAMBRIDGE ST,BOSTON,"736 Cambridge Street, Brighton, MA 02135",42.34958,-71.14798
21353,1730910134,PEREZ,LEDYS,4458 N 20TH ST,PHILADELPHIA,"4458 North 20th Street, Philadelphia, PA 19140",40.02254,-75.15832
18011,1578394052,MYERS,EVAN,780 AMERICAN LEGION HWY,BOSTON,"780 American Legion Highway, Roslindale, MA 02131",42.28101,-71.11536
2171,1659691491,PAGE,MARISSA,6 WELLNESS WAY STE 113,LATHAM,"6 Wellness Way, Latham, NY 12110",42.76665,-73.76008
3638,1720818784,CARRANZA,KYLEE,200 W SANTA ANA BLVD STE 801,SANTA ANA,"200 West Santa Ana Boulevard, Santa Ana, CA 92701",33.74930,-117.86916
20380,1932930393,HASHMI,ROMAISA,14 FORDHAM RD,ALLSTON,"14 Fordham Road, Allston, MA 02134",42.35180,-71.12766
